<a href="https://colab.research.google.com/github/oranzoni/stock-prediction/blob/main/Logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Sad treba uraditi drugi neki tip klasifikacije ali bi se mogli mozda dodati neki features koji ce se proracunati
#Prvo treba ucitati gotov dataset u koji je dodan osnovni feature
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [6]:
df_ready = pd.read_csv('/content/konacni_dataset_R_label.csv')
df_ready

,Unnamed: 0,Ticker,Date,Close,High,Low,Open,Volume,Open tomorrow,Return value,Return target
0,0,A,2021-01-04,115.337715,116.747352,113.850303,115.629368,2.030700e+06,115.182172,-0.003867,0.0
1,1,AA,2021-01-04,22.343918,23.497153,22.223789,22.699499,5.400000e+06,22.391968,-0.013548,0.0
2,2,AAL,2021-01-04,15.130000,15.880000,15.030000,15.850000,6.973250e+07,14.920000,-0.058675,0.0
3,3,AAON,2021-01-04,42.214161,43.936391,41.940171,43.649352,4.200000e+05,42.070645,-0.036168,0.0
4,4,AAP,2021-01-04,141.919510,143.930963,140.674769,142.424624,8.491000e+05,140.918299,-0.010576,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1065247,1065247,ZION,2025-05-29,47.730000,47.750000,46.889999,47.279999,1.421500e+06,NaN,NaN,NaN
1065248,1065248,ZM,2025-05-29,79.940002,80.309998,78.989998,80.230003,1.906400e+06,NaN,NaN,NaN
1065249,1065249,ZS,2025-05-29,251.110001,255.160004,248.740005,255.000000,4.228500e+06,NaN,NaN,NaN
1065250,1065250,ZTS,2025-05-29,167.139999,167.789993,165.110001,165.919998,2.450400e+06,NaN,NaN,NaN


In [5]:
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
!pip install conda-package-handling
!wget https://anaconda.org/conda-forge/ta-lib/0.5.1/download/linux-64/ta-lib-0.5.1-py311h9ecbd09_0.conda
!cph x ta-lib-0.5.1-py311h9ecbd09_0.conda
!mv ./ta-lib-0.5.1-py311h9ecbd09_0/lib/python3.11/site-packages/talib /usr/local/lib/python3.11/dist-packages/


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4039    0  4039    0     0   2825      0 --:--:--  0:00:01 --:--:--  2824
100  517k  100  517k    0     0   287k      0  0:00:01  0:00:01 --:--:--  287k
--2025-06-02 17:18:03--  https://anaconda.org/conda-forge/ta-lib/0.5.1/download/linux-64/ta-lib-0.5.1-py311h9ecbd09_0.conda
Resolving anaconda.org (anaconda.org)... 104.19.144.37, 104.19.145.37, 2606:4700::6813:9025, ...
Connecting to anaconda.org (anaconda.org)|104.19.144.37|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://binstar-cio-packages-prod.s3.amazonaws.com/5f738b4b78d4370a69f82984/673b62920c609ff6945bb24d?response-content-disposition=attachment%3B%20filename%3D%22ta-lib-0.5.1-py311h9ecbd09_0.conda%22%3B%20filename%2A%3DUTF-8%27%27ta-lib-0.5.1-py311h9ecbd09_0.conda&response-content-type=application%2Foctet-stream&X-Amz-Algorithm=

In [7]:
import talib as ta
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score

In [8]:
#prvo će trebati dodati features koje su korisne za logistic regression, a potom uraditi treniranje na osnovu tih features
#moving average crossover
#RSI - relative strength index - speed and change of price movements of a stock
#volume - to već imamo
#koristit će se RSI, ali treba naznaciti da direktno ne korelira sa opening price, ali predstavlja dobar generalni indikator kretanja dionica
#RSI je zapravo momentum koji se izracunava na osnovu cijene zatvaranja
df_ready["RSI"] = ta.RSI(np.array(df_ready["Close"]), timeperiod =14) #obicno se uzima 14 dana za RSI kalkulaciju
df_ready["MA_21"] = ta.SMA(df_ready["Open"])
df_ready["MA_5"] = ta.SMA(df_ready["Open"])


In [9]:
df_ready.to_csv('LR_dataset_full.csv')

In [10]:
model_lr = LogisticRegression(penalty = "l2", solver = "newton-cg", max_iter=100000)
#Posto LR ne prima NaN values, morat ce se ukloniti
df_ready = df_ready.dropna()

x_train = df_ready.iloc[:-200000].drop("Return target", axis=1)
y_train = df_ready.iloc[:-200000]["Return target"]

x_test  = df_ready.iloc[-200000:].drop("Return target", axis=1)
y_test  = df_ready.iloc[-200000:]["Return target"]


In [11]:
predictors = ["Volume", "RSI", "Open", "Close"]
model_lr.fit(x_train[predictors], y_train)

/usr/local/lib/python3.11/dist-packages/scipy/optimize/_linesearch.py:312: LineSearchWarning: The line search algorithm did not converge
  alpha_star, phi_star, old_fval, derphi_star = scalar_search_wolfe2(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/optimize.py:100: LineSearchWarning: The line search algorithm did not converge
  ret = line_search_wolfe2(
/usr/local/lib/python3.11/dist-packages/scipy/optimize/_linesearch.py:312: LineSearchWarning: Rounding errors prevent the line search from converging
  alpha_star, phi_star, old_fval, derphi_star = scalar_search_wolfe2(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/optimize.py:312: UserWarning: Line Search failed
  warnings.warn("Line Search failed")


LogisticRegression(max_iter=100000, solver='newton-cg')

In [12]:
from sklearn.metrics import precision_score
predictions = model_lr.predict(x_test[predictors])
predictions= pd.Series(predictions, index = x_test.index, name= 'Return predikcije')

In [13]:
predictions

,Return predikcije
864256,0.0
864257,0.0
864258,1.0
864259,1.0
864260,1.0
...,...
1064251,0.0
1064252,1.0
1064253,0.0
1064254,0.0


In [14]:
precision_modela = precision_score(y_test, predictions)
precision_modela

0.8266258493242739

Zaključak: potrebno je mnogo više iteracija (100000 minimalno ) za neki pristojan precision score. Uz to, "newton-cg" solver uz "l2" se pokazao kao zadovoljavajući. "newton-cg" solver zasniva se na Newton-Rhapsonovom metodu, brzo konvergira i pronalazi rješenje. l2 je ridge optimizacija koja isto brzo dovodi do rješenja za velike datasetove - DORADITI SVA OVA OBJAŠNJENJA

In [15]:
#Recall - number of true positives in the model; considered good for 0.75 or above
from sklearn.metrics import recall_score
recall_modela = recall_score(y_test, predictions)
recall_modela

0.8616905354919053

In [16]:
F1 = 2* (precision_modela * recall_modela) / (precision_modela + recall_modela)
F1

0.843794062726268

BACKTESTING prema zahtjevima zadatka: uvodi se nova kolona u kojoj će se računati : 1*Return target ako je predikcija == 1 ili -1*Return target ako je predikcija 0. Na kraju se to sve treba sumirati i vidjeti koliki je profit

Alternativa za ovo jeste da se napravi confidence trashold - za logistic regression je dobro uzeti da je 0.5, to je neki default
Treba uraditi sljedece: proracunati s proba funkcijom vjerovatnocu da ce predicted return biti 1 ili 0, pa onda uzeti treshold i vidjeti gdje prelazi probability 50 posto. Kad se to uradi, onda isto uraditi kao za prethodni slucaj : 1*Return ako je predikcija == 1 ili -1*Return ako je predikcija 0 i sumirati

DAILY TRADING RULE 1.1

In [17]:
#za ovo je lakse spojiti x_test i y_test u jedan dataframe, pa cu to i uraditi, jer ce se imati slika svih kolona koje su potrebne
#inace su bili odvojeni za logistic regression, iako je u principu sve isto kao i kod random foresta
# bitno je samo sta koji df ili pandas series sadrzi - sto se vidi da je isto kod oba slucaja - notacija je drugacija

df_backtest = pd.concat([df_ready.iloc[-200000:], predictions], axis = 1) #treba da bude sve sadrzano, predikcije i stvarni testni podaci
df_backtest["Backtest_1"] = np.where(df_backtest["Return predikcije"] == df_backtest["Return target"], "1","-1")


In [18]:
total_return_daily = df_backtest.groupby("Date")["Backtest_1"].agg(
    lambda grp: sum(int(v) for v in grp)
)

In [19]:
total_return_daily

,Backtest_1
Date,
2024-08-05,311
2024-08-06,641
2024-08-07,749
2024-08-08,899
2024-08-09,725
...,...
2025-05-21,882
2025-05-22,-14
2025-05-23,810


DAILY TRADING RULE 1.1 - CONFIDENCE TRESHOLD

In [20]:
probabilities_thr = model_lr.predict_proba(x_test[predictors])
threshold = 0.5
predictions_thres = (probabilities_thr[:, 1] > threshold).astype(int)
predictions_thres


array([0, 0, 1, ..., 0, 0, 0])

In [21]:
df_backtest["Backtest_2"] = np.where(df_backtest["Return predikcije"] == predictions_thres, "1","-1")

In [22]:
daily_returns_int = df_backtest.groupby("Date")["Backtest_2"].agg(
    lambda grp: sum(int(v) for v in grp)
)

In [23]:
daily_returns_int

,Backtest_2
Date,
2024-08-05,359
2024-08-06,989
2024-08-07,989
2024-08-08,989
2024-08-09,989
...,...
2025-05-21,996
2025-05-22,996
2025-05-23,996


2. IN BACKTESTING - PORTFOLIO RETURN

In [34]:
portfolio_daily = df_backtest.groupby(["Ticker", "Date"])["Return value"].mean()
portfolio_daily

Ticker  Date      
A       2024-08-06    0.013513
        2024-08-07   -0.022587
        2024-08-08    0.020052
        2024-08-09   -0.000219
        2024-08-12   -0.007163
                        ...   
^GSPC   2025-05-21   -0.011661
        2025-05-22   -0.010164
        2025-05-23    0.012484
        2025-05-27    0.012209
        2025-05-28    0.002434
Name: Return value, Length: 200000, dtype: float64